# Keras intro exercises

## 1. Build a simple sequential model

* Can you build a sequential model to reproduce the graph shown in the figure? 
* Assume that this is a classifier
* Choose whatever activations you want, wherever possible
* How many classes are we predicting?

<center><img src="figures/sequence_api_exercise.png"></center>

In [ ]:
#!wget https://raw.githubusercontent.com/NBISweden/workshop-neural-nets-and-deep-learning/master/session_annBuildingBlocks/lab_keras/exercises.ipynb
!mkdir figures
!wget -P figures/ https://raw.githubusercontent.com/NBISweden/workshop-neural-nets-and-deep-learning/master/session_annBuildingBlocks/lab_keras/figures/sequence_api_exercise.png

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

#Add your model here
model = Sequential()
...
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, "figures/exercise_model.png", show_shapes=True)

## 2. Build a better XOR classifier

Given the model seen at lecture, how do we make a better classifier (higher accuracy)?

* More layers? More neurons?
* Generate more data?
* More epochs?
* Different batch size?
* Different optimizer?
* It's up to you! Let's see who does best on validation

Only for Tuesday's session:

* Different activations?
* Add Dropout? How large?

Training curve plotting function:

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

def plot_loss_acc(history):
    try:
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
    except:
        plt.plot(history.history['acc'])
        plt.plot(history.history['val_acc'])
        
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train acc', 'val acc', 'train loss', 'val loss'], loc='upper left')
    plt.show()

Data generation step:

In [ ]:
# Generate XOR data
data = np.random.random((10000, 3)) - 0.5
labels = np.zeros((10000, 1))

labels[np.where(np.logical_xor(np.logical_xor(data[:,0] > 0, data[:,1] > 0), data[:,2] > 0))] = 1

#let's print some data and the corresponding label to check that they match the table above
for x in range(3):
    print("{0: .2f} xor {1: .2f} xor {2: .2f} equals {3:}".format(data[x,0], data[x,1], data[x,2], labels[x,0]))


The baseline network to improve:

In [ ]:
model = Sequential()
model.add(Dense(4, input_dim=3, activation='tanh'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Train the model, iterating on the data in batches of 32 samples
history = model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.1)

In [ ]:
#Add your model here

In [ ]:
plot_loss_acc(history)

## 3. Build a regression model

* Take the Boston housing dataset (http://lib.stat.cmu.edu/datasets/boston)
* Records a set of variables for a set of houses in Boston, including among others:
    * CRIM     per capita crime rate by town
    * ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
    * INDUS    proportion of non-retail business acres per town
    * CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
    * NOX      nitric oxides concentration (parts per 10 million)
    * RM       average number of rooms per dwelling
* Can we use these variables to predict the value of a house (in tens of thousands of dollars)?

In [ ]:
import tensorflow
from sklearn.preprocessing import StandardScaler #hint
from keras.layers import Dropout
#This is how we load the dataset, pre-split in training/validation sets
(X_train, y_train), (X_val, y_val) = tensorflow.keras.datasets.boston_housing.load_data(path="boston_housing.npz", test_split=0.2, seed=113)

#Let's have a look at the data
print(X_train.shape)
print(X_train[0], y_train[0])


# Add your model here
model = Sequential()

model.compile(...)

history = model.fit(...)

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

def plot_loss_mae(history, metric):
    
    fig,ax = plt.subplots()
    ax.plot(history.history[metric])
    ax.plot(history.history['val_' + metric])
    ax.set_ylabel(metric)
    ax2=ax.twinx()
    ax2.plot(history.history['loss'], c="red")
    ax2.plot(history.history['val_loss'], c="green")
    ax2.set_ylabel("loss")
    plt.title('model accuracy')
    #plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train ' + metric, 'val ' + metric, 'train loss', 'val loss'], loc='upper left')
    plt.show()
    
plot_loss_mae(history, "mean_absolute_error")

## 4. The IMDB movie review sentiment dataset

Another pre-package toy dataset from Keras. Contains 25k reviews for a movies in IMDB, you want to predict whether the review is positive or negative.

> each review is encoded as a list of word indexes (integers). For convenience, words are indexed by overall frequency in the dataset, so that for instance the integer "3" encodes the 3rd most frequent word in the data. This allows for quick filtering operations such as: "only consider the top 10,000 most common words, but eliminate the top 20 most common words".

https://keras.io/api/datasets/imdb/

Load the dataset, set a couple of important parameters (max_features, maxlen). Also pad all reviews with less than 200 words so that they have all the same length.

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

max_features = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review

(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(
    num_words=max_features
)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen, padding="post")
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen, padding="post")


Since the dataset is pre-processed so that each word is represented by an integer, we have to build a reverse dictionary if we want to actually read some of the reviews:

In [ ]:
# Use the default parameters to keras.datasets.imdb.load_data
start_char = 1
oov_char = 2
index_from = 3
# Retrieve the word index file mapping words to indices
word_index = keras.datasets.imdb.get_word_index()
# Reverse the word index to obtain a dict mapping indices to words
# And add `index_from` to indices to sync with `x_train`
inverted_word_index = dict(
    (i + index_from, word) for (word, i) in word_index.items()
)

inverted_word_index[0] = ""
# Update `inverted_word_index` to include `start_char` and `oov_char`
inverted_word_index[start_char] = "[START]"
inverted_word_index[oov_char] = "[OOV]"
# Decode the first sequence in the dataset


In [ ]:
print([i for i in x_train[2]])

In [ ]:
decoded_sequence = " ".join(inverted_word_index[i] for i in x_train[2])
print(decoded_sequence)

How do we build a predictor for this task?

In [ ]:
model = Sequential()
...